# Problem definition

When the following conditions are met:

- The domain can be fully observed
- The range of possible actions is known
- There's a finite set of actions to select from
- The domain operates deterministically
- The domain remains static with only our actions creating changes

it becomes possible to model a **problem**, conceptualize it as a **search problem**, and utilize various search algorithms to reach the goal.

In [3]:
class Problem:
    def __init__(self, initial, goal=None):
        # The goal state is optional, because some problems may not have a goal state.
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        # Given a state, return a list of possible actions.
        raise NotImplementedError

    def result(self, state, action):
        # Given a state and an action, return the new state that results from applying the action to the state.
        raise NotImplementedError

    def goal_test(self, state):
        # Return True if the state is a goal state.
        return state == self.goal

    def path_cost(self, c, state1, action, state2):
        # Return the cost of the path that arrives at state2 from state1 via action, assuming cost c to get up to state1.
        return c + 1

# State Space

Search algorithms are designed to locate a specific target state. But they also need to access the state space, including all possible states the agent could potentially be in.

The solution of the search problem is a **path** from initial state to goal state: `I → A → S → G`. We simplify the task and store a `Node`. Each node has a parent node, so the path can be rebuilt if necessary.

Reference implementation: https://github.com/aimacode/aima-python/blob/master/search.py

In [4]:
class Node:
    def __init__(self, state, action=None, parent=None, path_cost=0) -> None:
        self.state = state
        self.cost = path_cost
        self.action = action
        self.parent = parent

    def __repr__(self) -> str:
        return f"<Node state: {self.state}, cost: {self.cost}>"

    # Expand the frontier.
    def expand(self, problem):
        return [
            self.child_node(problem, action) for action in problem.actions(self.state)
        ]

    # Get child node based on the action.
    def child_node(self, problem, action):
        next_state = problem.result(self.state, action)
        next_node = Node(
            next_state,
            self,
            action,
            problem.path_cost(self.path_cost, self.state, action, next_state),
        )
        return next_node

    def path(self):
        node, path_back = self, []
        while node:
            path_back.append(node)
            node = node.parent
        return path_back[::-1]

    def __lt__(self, other):
        return self.cost < other.cost

In [5]:
# Simplified road map of Romania
romania_map = {
    "Arad": {"Zerind": 75, "Sibiu": 140, "Timisoara": 118},
    "Zerind": {"Arad": 75, "Oradea": 71},
    "Sibiu": {"Arad": 140, "Fagaras": 99, "Oradea": 151, "Rimnicu": 80},
    "Timisoara": {"Arad": 118, "Lugoj": 111},
    "Bucharest": {"Urziceni": 85, "Pitesti": 101, "Giurgiu": 90, "Fagaras": 211},
    "Urziceni": {"Bucharest": 85, "Hirsova": 98, "Vaslui": 142},
    "Pitesti": {"Bucharest": 101, "Craiova": 138, "Rimnicu": 97},
    "Giurgiu": {"Bucharest": 90},
    "Fagaras": {"Bucharest": 211, "Sibiu": 99},
    "Craiova": {"Drobeta": 120, "Rimnicu": 146, "Pitesti": 138},
    "Drobeta": {"Craiova": 120, "Mehadia": 75},
    "Rimnicu": {"Craiova": 146, "Pitesti": 97, "Sibiu": 80},
    "Mehadia": {"Drobeta": 75, "Lugoj": 70},
    "Eforie": {"Hirsova": 86},
    "Hirsova": {"Eforie": 86, "Urziceni": 98},
    "Iasi": {"Vaslui": 92, "Neamt": 87},
    "Vaslui": {"Iasi": 92, "Urziceni": 142},
    "Neamt": {"Iasi": 87},
    "Lugoj": {"Timisoara": 111, "Mehadia": 70},
    "Oradea": {"Zerind": 71, "Sibiu": 151},
}

Use a `Problem` interface to formulate search problem.

In [6]:
class PathFinder(Problem):
    def __init__(self, initial, goal, graph):
        super().__init__(initial, goal)
        self.graph = graph

    def actions(self, state):
        return self.graph[state].keys()

    def result(self, state, action):
        # The result is the name of the city we reach by taking the action.
        return action

    def path_cost(self, c, state1, action, state2):
        return c + self.graph[state1][state2]

    # No need to override goal_test method because it already does what we want.
    # def goal_test(self, state)

# Breadth First Search

In [32]:
from collections import deque


def breadth_first_graph_search(problem):
    frontier = deque([Node(problem.initial)])
    # It is important to track what is in the frontier and do not insert any nodes that are already there.
    frontier_states = set([problem.initial])
    explored = set()
    while frontier:
        node = frontier.pop()
        explored.add(node.state)
        frontier_states.remove(node.state)
        if problem.goal_test(node.state):
            return node
        for neighbor in problem.actions(node.state):
            if neighbor not in explored and neighbor not in frontier_states:
                neighbor_node = Node(
                    state=neighbor,
                    parent=node,
                    action=neighbor,
                    path_cost=problem.path_cost(node.cost, node.state, None, neighbor),
                )
                frontier.appendleft(neighbor_node)
                frontier_states.add(neighbor)

    return None

In [33]:
from pprint import pprint

problem = PathFinder("Arad", "Bucharest", romania_map)
solution = breadth_first_graph_search(problem)
pprint(solution.path())

[<Node state: Arad, cost: 0>,
 <Node state: Sibiu, cost: 140>,
 <Node state: Fagaras, cost: 239>,
 <Node state: Bucharest, cost: 450>]


The outcome is inaccurate because we didn't take into account the edge's weight (the road's length) when adding them to the frontier. This approach works well if all edges have the same cost, such as "1" in an unweighted graph. However, when we start considering the weights of the edges, we transition to a "Uniform Cost Search".

# Depth First Search

In [9]:
def depth_first_graph_search(problem):
    pass

# Cheapest First Search (Uniform Cost Search)

In [44]:
# from collections import deque
# Replacing the frontier from a deque to minheap.
import heapq


def uniform_cost_search(problem):
    frontier = [Node(problem.initial)]  # deque([Node(problem.initial)])
    # It is important to note that we can come to the same state/node with different costs.
    frontier_states = set([(problem.initial, 0)])
    heapq.heapify(frontier)
    explored = set()
    while frontier:
        node = heapq.heappop(frontier)  # frontier.pop()
        explored.add(node.state)
        frontier_states.remove((node.state, node.cost))
        if problem.goal_test(node.state):
            return node
        for neighbor in problem.actions(node.state):
            if neighbor not in explored and neighbor not in frontier_states:
                cost = problem.path_cost(node.cost, node.state, None, neighbor)
                neighbor_node = Node(
                    state=neighbor,
                    parent=node,
                    action=neighbor,
                    path_cost=cost,
                )
                heapq.heappush(
                    frontier, neighbor_node
                )  # frontier.appendleft(neighbor_node)
                # print(f"heappush: {neighbor_node}, {cost}")
                frontier_states.add((neighbor, cost))

    return None

In [45]:
from pprint import pprint

problem = PathFinder("Arad", "Bucharest", romania_map)
solution = uniform_cost_search(problem)
solution.path()

[<Node state: Arad, cost: 0>,
 <Node state: Sibiu, cost: 140>,
 <Node state: Rimnicu, cost: 220>,
 <Node state: Pitesti, cost: 317>,
 <Node state: Bucharest, cost: 418>]

Now we got the correct result!

# A* Search

In [ ]:
from dataclasses import dataclass


@dataclass(frozen=True)
class Location:
    __slots__ = ["x", "y"]
    x: int
    y: int


# We will need locations for a heuristic.
locations = {
    "Arad": Location(91, 492),
    "Bucharest": Location(400, 327),
    "Craiova": Location(253, 288),
    "Drobeta": Location(165, 299),
    "Eforie": Location(562, 293),
    "Fagaras": Location(305, 449),
    "Giurgiu": Location(375, 270),
    "Hirsova": Location(534, 350),
    "Iasi": Location(473, 506),
    "Lugoj": Location(165, 379),
    "Mehadia": Location(168, 339),
    "Neamt": Location(406, 537),
    "Oradea": Location(131, 571),
    "Pitesti": Location(320, 368),
    "Rimnicu": Location(233, 410),
    "Sibiu": Location(207, 457),
    "Timisoara": Location(94, 410),
    "Urziceni": Location(456, 350),
    "Vaslui": Location(509, 444),
    "Zerind": Location(108, 531),
}